In [1]:
import cv2,os

path=(r"C:\Users\arunf\OneDrive\Documents\Data Science\MASK DETECTION\DATA\data") #loading file
categories=os.listdir(path)       #get the folders or files
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels)) #empty dictionary

print(label_dict)
print(categories)
print(labels)

{'without_mask': 0, 'with_mask': 1}
['without_mask', 'with_mask']
[0, 1]


In [2]:
size=100
data=[]
target=[]

for folder in categories:
    folder_path=os.path.join(path,folder)
    img_names=os.listdir(folder_path)#files in folder
    
    for image in img_names:
        img_path=os.path.join(folder_path,image)
        img=cv2.imread(img_path)
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #gray image 
        resized=cv2.resize(gray,(size,size))#converting to similiar image
        data.append(resized)
        target.append(label_dict[folder])#list of 0 and 1
#print(target)

In [3]:
import numpy as np

data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],size,size,1))
target=np.array(target)



from keras.utils import np_utils
new_target=np_utils.to_categorical(target)
np.save('data',data)
np.save('target',new_target)

In [4]:
import numpy as np

data=np.load('data.npy')
target=np.load('target.npy')

In [5]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D

In [6]:
model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(50,activation='relu'))
model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [7]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)

In [8]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=5,callbacks=[checkpoint],validation_split=0.2)

Epoch 1/5
170/170 [==============================] - ETA: 0s - loss: 0.6231 - accuracy: 0.6483

INFO:tensorflow:Assets written to: model-001.model\assets


INFO:tensorflow:Assets written to: model-001.model\assets


170/170 [==============================] - 194s 1s/step - loss: 0.6231 - accuracy: 0.6483 - val_loss: 0.5228 - val_accuracy: 0.7228
Epoch 2/5
170/170 [==============================] - ETA: 0s - loss: 0.4519 - accuracy: 0.7925

INFO:tensorflow:Assets written to: model-002.model\assets


...

INFO:tensorflow:Assets written to: model-004.model\assets


170/170 [==============================] - 195s 1s/step - loss: 0.3433 - accuracy: 0.8490 - val_loss: 0.3336 - val_accuracy: 0.8471
Epoch 5/5
170/170 [==============================] - ETA: 0s - loss: 0.3002 - accuracy: 0.8718

INFO:tensorflow:Assets written to: model-005.model\assets


INFO:tensorflow:Assets written to: model-005.model\assets


170/170 [==============================] - 193s 1s/step - loss: 0.3002 - accuracy: 0.8718 - val_loss: 0.3004 - val_accuracy: 0.8603


In [9]:
print(model.evaluate(test_data,test_target))

24/24 [==============================] - 5s 228ms/step - loss: 0.3443 - accuracy: 0.8333
[0.3443070650100708, 0.8333333134651184]


In [10]:
from keras.models import load_model

model.save('model-005.model')

INFO:tensorflow:Assets written to: model-005.model\assets


INFO:tensorflow:Assets written to: model-005.model\assets


In [11]:
#from tensorflow.python.keras.models import load_model
import os
import cv2
import numpy as np
from keras.models import load_model

In [12]:
model = load_model('model-005.model')

In [13]:
face_clsfr=cv2.CascadeClassifier(r"C:\Users\arunf\OneDrive\Documents\Data Science\AI\haarcascade_frontalface_default.xml")
source=cv2.VideoCapture(0)
labels_dict={1:'With Mask',0:'Without Mask'}
color_dict={1:(0,255,0),0:(0,0,255)}

In [ ]:
while(True):
    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_clsfr.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        
    )   
    for (x,y,w,h) in faces:
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        result=model.predict(reshaped)
        
        label=np.argmax(result,axis=1)[0]
        
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-50),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    cv2.imshow('Mask Detection',img)
    key=cv2.waitKey(1)
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()

1/1 [==============================] - 0s 25ms/step
